In [1]:
import os
import csv
import pandas as pd
import numpy as np
import umap
import hdbscan
import matplotlib.pyplot as plt
import data
from sklearn.feature_extraction.text import CountVectorizer
from sentence_transformers import SentenceTransformer
from  sklearn.metrics.pairwise import cosine_similarity
from random import sample

In [2]:
data_path = "data/negotiate"
lines = []
file_name = os.path.join(data_path, 'data.txt')
with open(file_name, 'r') as f:
    for line in f:
        lines.append(line.strip())
g =  [line[:12] for line in lines]
o =  [line[-12:] for line in lines]
selection = [line[12:-12].split('<eos>')[-2].split('<selection>')[1] for line in lines]
rew = [line[12:-12].split('<eos>')[-1][8:] for line in lines]
dialog = [line[12:-12].split('<eos>')[:-2] for line in lines]

In [25]:
partial = pd.read_csv("dataset_clustered.csv", index_col = 0)

NameError: name 'partial' is not defined

In [27]:
unannot = partial[partial['annotated'] == False]

In [29]:
model = SentenceTransformer('distilbert-base-nli-mean-tokens')
embeddings = model.encode(list(unannot['sentences']), show_progress_bar=True)

Batches:   0%|          | 0/763 [00:00<?, ?it/s]

In [30]:
umap_embeddings = umap.UMAP(n_neighbors=15, 
                            n_components=5, 
                            metric='cosine').fit_transform(embeddings)

In [31]:
cluster = hdbscan.HDBSCAN(min_cluster_size=60, min_samples =5,
                          metric='euclidean',                      
                          cluster_selection_method='eom').fit(umap_embeddings)

In [32]:
reclust = pd.DataFrame({"sentences": unannot['sentences'], "cluster2":cluster.labels_})

In [33]:
partial = partial.join(reclust['cluster2'])

In [34]:
partial.loc[partial['cluster2'].isna(), 'cluster2'] = -1

In [35]:
partial['cluster2'].value_counts()

-1.0     10283
 0.0      1328
 87.0     1159
 8.0       986
 81.0      937
         ...  
 86.0       62
 21.0       61
 18.0       60
 89.0       60
 75.0       60
Name: cluster2, Length: 97, dtype: int64

In [39]:
partial[partial['cluster2'] == 8]

,sentences,greeting,need:books,need:hats,need:balls,offer:books,offer:hats,offer:balls,inquiry,agreement,thanks,same,annotated,cluster,cluster2
index,,,,,,,,,,,,,,,
25712,"1 ) one book and one hat , or 2 ) two hats .",False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,20,8.0
3369,1 book 2 hats,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,20,8.0
1097,"1 hat , 2 books",False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,20,8.0
28798,2 balls and 1 book and we have a deal,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,20,8.0
7681,2 balls and the book . and i`ll take 2 hats,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,20,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26124,"you take hat and two books , i'll take the rest .",False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,20,8.0
25605,you take two books and i keep the rest,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,20,8.0
12138,you want 2 hats and a book correct,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,20,8.0


In [38]:
(partial[partial['cluster2'] == 87].sentences).value_counts()

i need the book or no deal                                             5
i need book or no deal                                                 3
i can not make that deal . i need the book or i can not make a deal    2
i need the book or i can not make a deal .                             2
i can't give up the book                                               2
                                                                      ..
i cant make a deal unless i get that book                              1
i cant use any of the books                                            1
i cant use the books                                                   1
i could not give up both books                                         1
youre being complicated you said the book is the onlything of value    1
Name: sentences, Length: 1148, dtype: int64

In [40]:
(partial[partial['cluster2'] == 0].sentences).value_counts()

you can have 3 balls                                                                                   4
how about you get three books                                                                          3
i need the ball and three books                                                                        2
i want the book and 3 hats                                                                             2
3 books                                                                                                2
                                                                                                      ..
i need books , so i guess i can live with your first offer - you take 3 hats and i'll take the rest    1
i need book and three booksw                                                                           1
i need at least three to be worth it                                                                   1
i need at least three objects .                        

In [41]:
(partial[partial['cluster2'] == 1].sentences).value_counts()

what about basketballs ?                                                                                                                          2
i would like the basketball                                                                                                                       2
i would like the hat and all the books you can have the basketball                                                                                2
i would like the hat and basketball you can have all the books                                                                                    2
i'd like the hats and the basketball ; are the books of high value to you ?                                                                       2
                                                                                                                                                 ..
i really like the book . i'll give you the basketball if i can have everything else .                           

In [42]:
(partial[partial['cluster2'] == 2].sentences).value_counts()

give me hat and balls                                7
balls and hats for me                                6
hats and balls for me                                5
give me hat and ball                                 4
can i have the hat and balls ?                       3
                                                    ..
hat and ball                                         1
give the balls and hats to me                        1
give me the hats and balls                           1
give me the hats and ball                            1
you take the balls , i take the hats and the ball    1
Name: sentences, Length: 315, dtype: int64

In [43]:
(partial[partial['cluster2'] == 3].sentences).value_counts()

correct                      14
okay agreed .                12
very well                     5
agreed !                      4
correct .                     3
true                          3
that's correct                2
okay , agreed .               2
agreed , yes                  1
yes agreed                    1
yes agree                     1
yes , that's correct .        1
very well . .                 1
very well .                   1
thats correct                 1
sounds about right            1
that is very true .           1
that is correct .             1
right                         1
okay agreed                   1
okay , that's agreeable .     1
ok . agreed .                 1
ok . agreed !                 1
correct ,                     1
confirmed                     1
yes that is correct .         1
Name: sentences, dtype: int64

In [ ]:
(partial[partial['cluster2'] == 4].sentences).value_counts()

In [ ]:
(partial[partial['cluster2'] == 5].sentences).value_counts()

In [ ]:
(partial[partial['cluster2'] == 6].sentences).value_counts()

In [ ]:
(partial[partial['cluster2'] == 7].sentences).value_counts()

In [23]:
(partial[partial['cluster2'] == 8].sentences).value_counts()

,sentences,greeting,need:books,need:hats,need:balls,offer:books,offer:hats,offer:balls,inquiry,agreement,thanks,same,annotated,cluster,cluster2
index,,,,,,,,,,,,,,,
12316,1,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,16,2.0
7922,2,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,19,-1.0
13776,2,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,19,-1.0
21820,2,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,19,-1.0
1611,3,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,7,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19122,yup !,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Full,False,False,True,5,-1.0
20381,yup impossible unless one person gets low poin...,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,16,2.0
16405,yup no deal,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,-1,2.0
